In [0]:
# SparkSQL imports
import pyspark.sql.functions as f
from pyspark.sql import Row
from pyspark.sql.types import *

# SparkML imports
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, Bucketizer, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

from datetime import datetime

# Establish Spark context
sc = spark.sparkContext 

In [0]:
# %fs rm -r /FileStore/tables/uc_past_patients.csv

res2: Boolean = true

In [0]:
patients_file = "dbfs:///FileStore/tables/uc_past_patients.csv"

patients_df =  spark.read.format('csv') \
    .option('header', True) \
    .load(patients_file)

In [0]:
patients_df = patients_df.filter(f.col('visit_date') == '2021-05-01')
patients_df.display()

pt_id,pt_name,pt_dob,pt_age,visit_location,visit_reason,visit_code,visit_date,visit_day,checkin_time,checkout_time,rolling_ct,rolling_code,num_techs,datetime
1000019,Jeffrey Landry,1971-09-17,50,denver,covid-test,4,2021-05-01,Saturday,08:34:29,08:56:29,1,0.0,3,2021-05-01 08:34:29
1000009,Laura Anderson,1971-01-20,51,denver,blood/lab-work,3,2021-05-01,Saturday,09:22:58,09:55:58,1,0.0,3,2021-05-01 09:22:58
1000010,Paul Hammond,1994-05-20,27,denver,headache,5,2021-05-01,Saturday,09:38:04,10:52:04,2,3.0,3,2021-05-01 09:38:04
1000011,Meghan Richards,1973-04-15,49,denver,vaccination,3,2021-05-01,Saturday,09:59:33,11:12:33,2,5.0,3,2021-05-01 09:59:33
1000002,Joshua Oliver,2005-06-22,16,denver,vaccination,3,2021-05-01,Saturday,10:00:05,11:02:05,3,4.0,3,2021-05-01 10:00:05
1000016,Anthony Johnson,1965-01-09,57,denver,UTI,4,2021-05-01,Saturday,10:35:10,11:24:10,1,0.0,3,2021-05-01 10:35:10
1000012,Jennifer Smith,1982-07-13,39,denver,covid-test,4,2021-05-01,Saturday,10:48:14,11:44:14,2,4.0,3,2021-05-01 10:48:14
1000004,Christopher Clark Jr.,1992-10-07,29,denver,cold/flu/fever,4,2021-05-01,Saturday,10:49:21,12:17:21,3,4.0,3,2021-05-01 10:49:21
1000003,Erica Thompson,1990-02-04,32,denver,covid-test,4,2021-05-01,Saturday,10:53:40,11:55:40,4,4.0,3,2021-05-01 10:53:40
1000001,Kevin Howe,1983-05-20,38,denver,cold/flu/fever,4,2021-05-01,Saturday,10:54:37,12:19:37,5,4.0,3,2021-05-01 10:54:37


In [0]:
# # patients_df.select(f.col('datetime')).transform(lambda x : patients_df.select(f.col('datetime').fromisoformat(x)))
def convert_time(x):
    return datetime.fromisoformat(x)
convert_timeUDF = udf(convert_time, DateType())
    
patients_df.withColumn('newdt', convert_timeUDF('datetime'))

Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/serializers.py", line 473, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
/databricks/spark/python/pyspark/serializers.py in dumps(self, obj)
    472         try:
--> 473             return cloudpickle.dumps(obj, pickle_protocol)
    474         except pickle.PickleError:

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dumps(obj, protocol, buffer_callback)
     72             )
---> 73             cp.dump(obj)
     74             return file.getvalue()

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dump(self, obj)
    562         try:
--> 563             return Pickler.dump(self, obj)
    564         except RuntimeError as e:

TypeError: cannot pickle '_thread.RLock' object

During handling of the above exception, another exception occurred:

PicklingError                             Traceback (most recent call last)
<command-2193271910545041> in <module>
      4 con

In [0]:
patients_df = patients_df.withColumn('datetime', patients_df.select(f.to_timestamp(patients_df.datetime, 'yyyy-MM-dd HH:mm:ss').alias('datetime')))
patients_df.display()


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2193271910545042> in <module>
----> 1 patients_df = patients_df.withColumn('datetime', patients_df.select(f.to_timestamp(patients_df.datetime, 'yyyy-MM-dd HH:mm:ss').alias('datetime')))
      2 patients_df.display()

/databricks/spark/python/pyspark/sql/dataframe.py in withColumn(self, colName, col)
   2651         """
   2652         if not isinstance(col, Column):
-> 2653             raise TypeError("col should be Column")
   2654         return DataFrame(self._jdf.withColumn(colName, col._jc), self.sql_ctx)
   2655 

TypeError: col should be Column

In [0]:
### REPARTITION DATA BY CLINIC
patients_df.write \
    .format("csv") \
    .mode("overwrite") \
    .option("header", True) \
    .partitionBy('visit_location') \
    .save("FileStore/tables/partitioned_clinics/") 